<a href="https://colab.research.google.com/github/Bellu95/computer-science-s-project/blob/master/progettoCS_19/20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
from google.colab import files

# mount google drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import autotime # ipython-autotime
%load_ext autotime
import datetime as  dt
start= dt.datetime.now()

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 1.28 ms


In [ ]:
# Read loans lenders file
loans_lenders = pd.read_csv("/content/drive/My Drive/python/kiva/loans_lenders.csv")
loans_lenders.head()

,loan_id,lenders
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."
2,485000,"muc888, terrystl, richardandsusan8352, sherri4..."
3,486087,"muc888, james5068, rudi5955, daniel9859, don92..."
4,534428,"muc888, niki3008, teresa9174, mike4896, david7..."


time: 7.28 s


1) Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender.


In [ ]:
# Each row is converted from string to list
loans_lenders['lenders'] = loans_lenders['lenders'].apply(lambda x:x.replace(',','').split())
# The explode function allows to transform each element of a list-like to a row, replicating index values by loan_columns
loans_lenders_splitted = loans_lenders.explode('lenders').reset_index(drop=True)
loans_lenders_splitted.head(10)

,loan_id,lenders
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499
5,483693,karlheinz4543
6,483693,jerrydb
7,483693,paula8951
8,483693,gmct
9,483693,amra9383


time: 15.4 s


2) For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing

In [ ]:
# Read loans file
loans = pd.read_csv("/content/drive/My Drive/python/kiva/loans.csv")
# Transform 'planned_expiration_time' and 'disburse_time' in pandas datetime format
loans['planned_expiration_time']=pd.to_datetime(loans['planned_expiration_time']).dt.date
loans['disburse_time']=pd.to_datetime(loans['disburse_time']).dt.date
# Create column 'duration'
loans['duration']=loans['planned_expiration_time'] - loans['disburse_time']
# Show columns 'duration'
loans['duration'].head()

0   54 days
1   96 days
2   37 days
3   35 days
4   58 days
Name: duration, dtype: timedelta64[ns]

time: 38.2 s


3) Find the lenders that have funded at least twice 

In [ ]:
lenders_funded=loans_lenders_splitted.groupby('lenders')['loan_id'].count().reset_index(name = 'count_funded')
lenders_funded[lenders_funded['count_funded']>1].head()

,lenders,count_funded
0,000,40
1,00000,39
2,0002,70
4,0101craign0101,71
5,0132575,4


time: 8.13 s


In [ ]:
print('The exact number about lenders that have funded at least twice is: ', len(lenders_funded[lenders_funded['count_funded']>1]))

The exact number about lenders that have funded at least twice is:  901811
time: 91.8 ms


4) For each country, compute how many loans have involved that country as borrowers

In [ ]:
Loan_by_country=loans.groupby('country_name')['loan_id'].count().reset_index(name = 'loans_count')
Loan_by_country.head()

,country_name,loans_count
0,Afghanistan,2337
1,Albania,3075
2,Armenia,13952
3,Azerbaijan,10172
4,Belize,218


time: 85.3 ms


5) For each country, compute the overall amount of money borrowed

In [ ]:
loan_amount_sum_by_country=loans.groupby('country_name')['loan_amount'].sum().reset_index(name='loan_amount_sum')
loan_amount_sum_by_country.head()

,country_name,loan_amount_sum
0,Afghanistan,1967950.0
1,Albania,4307350.0
2,Armenia,22950475.0
3,Azerbaijan,14784625.0
4,Belize,150175.0


time: 87.4 ms


6) Like the previous point, but expressed as a percentage of the overall amount lent.

In [ ]:
loan_amount_sum_by_country_percentual=(loans.groupby('country_name')['loan_amount'].sum()/loans['funded_amount'].sum()*100).reset_index(name = 'loan_amount_%_over_total')
loan_amount_sum_by_country_percentual.head()

,country_name,loan_amount_%_over_total
0,Afghanistan,0.174126
1,Albania,0.381119
2,Armenia,2.030683
3,Azerbaijan,1.308160
4,Belize,0.013288


time: 89 ms


7) Like the three previous points, but split for each year (with respect to disburse time)

In [ ]:
# Create a new column corresponding to the year of disburse_time
loans['disburse_time_year']=pd.to_datetime(loans['disburse_time']).dt.year

time: 318 ms


In [ ]:
ES_4_for_each_year=loans.groupby(['country_name','disburse_time_year'])['loan_id'].count().reset_index(name='loan_count')
ES_4_for_each_year

,country_name,disburse_time_year,loan_count
0,Afghanistan,2007.0,408
1,Afghanistan,2008.0,370
2,Afghanistan,2009.0,678
3,Afghanistan,2010.0,632
4,Afghanistan,2011.0,247
...,...,...,...
743,Zimbabwe,2013.0,426
744,Zimbabwe,2014.0,2078
745,Zimbabwe,2015.0,600
746,Zimbabwe,2016.0,808


time: 161 ms


In [ ]:
ES_5_for_each_year=loans.groupby(['country_name','disburse_time_year'])['loan_amount'].sum().reset_index(name='loan_amount_sum')
ES_5_for_each_year

,country_name,disburse_time_year,loan_amount_sum
0,Afghanistan,2007.0,194975.0
1,Afghanistan,2008.0,365375.0
2,Afghanistan,2009.0,585125.0
3,Afghanistan,2010.0,563350.0
4,Afghanistan,2011.0,245125.0
...,...,...,...
743,Zimbabwe,2013.0,678525.0
744,Zimbabwe,2014.0,1311575.0
745,Zimbabwe,2015.0,723625.0
746,Zimbabwe,2016.0,788600.0


time: 120 ms


In [ ]:
ES_6_for_each_year=(loans.groupby(['country_name','disburse_time_year'])['loan_amount'].sum()/loans['funded_amount'].sum()*100).reset_index(name='loan_amount_%_over_total')
ES_6_for_each_year

,country_name,disburse_time_year,loan_amount_%_over_total
0,Afghanistan,2007.0,0.017252
1,Afghanistan,2008.0,0.032329
2,Afghanistan,2009.0,0.051772
3,Afghanistan,2010.0,0.049846
4,Afghanistan,2011.0,0.021689
...,...,...,...
743,Zimbabwe,2013.0,0.060037
744,Zimbabwe,2014.0,0.116050
745,Zimbabwe,2015.0,0.064027
746,Zimbabwe,2016.0,0.069776


time: 135 ms


8) For each lender, compute the overall amount of money lent. For each loan that has more than one lender, you must assume that all lenders contributed the same amount

In [ ]:
# For each loan, new columns have been created corresponding to the funded amount and loan amount by each lenders present in the aforementioned loan
loans['funded_amount_x_lenders']=loans['funded_amount']/loans['num_lenders_total']
loans['loan_amount_x_lenders']=loans['loan_amount']/loans['num_lenders_total']
# Merge between dataset (loans) and dataset (loand lenders splitted) 
merge=loans[['loan_id','funded_amount_x_lenders','loan_amount_x_lenders']].merge(loans_lenders_splitted)
# Compute the overall amount of money lent and the overall amount of money borrowed for each lender
merge=pd.DataFrame(merge.groupby('lenders')[['funded_amount_x_lenders','loan_amount_x_lenders']].sum())
merge.reset_index(inplace=True)
# Show the overall amount of monney lent for each lender
merge[['lenders','funded_amount_x_lenders']]

,lenders,funded_amount_x_lenders
0,000,1512.095370
1,00000,1249.127691
2,0002,2201.180463
3,00mike00,38.461538
4,0101craign0101,2424.088932
...,...,...
1383794,zzmcfate,57535.947017
1383795,zzpaghetti9994,44.247788
1383796,zzrvmf8538,480.818179
1383797,zzzsai,236.888048


time: 15 s


In [ ]:
loans[['loan_id','funded_amount_x_lenders','loan_amount_x_lenders']][loans['num_lenders_total']==0].merge(loans_lenders_splitted)

,loan_id,funded_amount_x_lenders,loan_amount_x_lenders,lenders


time: 1.81 s


9) For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.

In [ ]:
# Read lenders file
lenders = pd.read_csv("/content/drive/My Drive/python/kiva/lenders.csv")
# Drop rows with Nan values in 'country code' and 'permanent name' columns and rename the column permanent name
lenders.dropna(subset=['country_code'],inplace=True)
lenders.dropna(subset=['permanent_name'],inplace=True)
lenders.rename(columns={'permanent_name':'lenders'},inplace=True)
lenders.head()

,lenders,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited
16,naresh2074,Naresh,NaN,NaN,US,1461303119,NaN,NaN,3.0,NaN,0
31,christina27976796,Christina,NaN,NaN,US,1461303341,NaN,NaN,1.0,Peter Tan,0
37,vikas1098,Vikas,Bengaluru,NaN,IN,1461301846,Software Engineer,NaN,2.0,NaN,0
39,qian1385,Qian,NaN,NaN,US,1461302036,NaN,NaN,2.0,NaN,0
42,xigg8769,Darla,Madison,NaN,US,1461302503,NaN,NaN,NaN,NaN,0


time: 5.19 s


In [ ]:
# Merge between dataset (lenders dropped) and the dataset created at the previous point 
df=merge.merge(lenders[['lenders','country_code']])
# Compute the difference between loan_amount_x_lenders and funded_amount_x_lenders for each country
diff_country=(df.groupby('country_code')['loan_amount_x_lenders'].sum()-df.groupby('country_code')['funded_amount_x_lenders'].sum()).reset_index(name='diff_between_fundedamount_and_loanamount')
diff_country

,country_code,diff_between_fundedamount_and_loanamount
0,AD,62.087704
1,AE,43485.581793
2,AF,3623.762951
3,AG,0.000000
4,AI,0.000000
...,...,...
220,YE,42.500000
221,YT,0.000000
222,ZA,9289.940114
223,ZM,2616.493185


time: 2.14 s


10) Which country has the highest ratio between the difference computed at the previous point and the population?

In [ ]:
# Read file country stats
country_stats=pd.read_csv('/content/drive/My Drive/python/kiva/country_stats.csv')
# Merge between dataset created at the previous point and dataset (country_stats)
ratio_population=country_stats[['country_code','country_name','population']].merge(diff_country)
# Create a new column about ratio between the difference computed at the previous point and the population
ratio_population['ratio_diffFundedLoan_population']=ratio_population['diff_between_fundedamount_and_loanamount']/ratio_population['population']
# Show the country  with the highest ratio
ratio_population[ratio_population['ratio_diffFundedLoan_population']==ratio_population['ratio_diffFundedLoan_population'].max()]

,country_code,country_name,population,diff_between_fundedamount_and_loanamount,ratio_diffFundedLoan_population
156,NO,Norway,5305383,849455.417811,0.160112


time: 804 ms


11) Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line

In [ ]:
# Drop rows with Nan values in 'population_below_poverty_line' columns and rename the column permanent name
country_stats.dropna(subset=['population_below_poverty_line'],inplace=True)
# For each country, create a new column about difference between the total amount of population and the amount of population under the poverty line
country_stats['not_poor']=country_stats['population'] - (country_stats['population_below_poverty_line']*country_stats['population']/100)

time: 5.03 ms


In [ ]:
# Merge between dataset (country_stats_dropped) and dataset created at the point 9
ratio_population_rich=country_stats.merge(diff_country)
# Create a new column about ration between the difference computed at point 9 and the population that is not below the poverty line
ratio_population_rich['ratio']=ratio_population_rich['diff_between_fundedamount_and_loanamount'] /ratio_population_rich['not_poor']

ratio_population_rich[ratio_population_rich['diff_between_fundedamount_and_loanamount']==ratio_population_rich['diff_between_fundedamount_and_loanamount'].max()]

,country_name,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name,not_poor,diff_between_fundedamount_and_loanamount,ratio
6,United States,US,USA,Americas,Northern America,324459463,15.1,0.919553,79.222,16.53815,13.21789,53245.07696,United States,2.754661e+08,1.849178e+07,0.067129


time: 31.5 ms


12) For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year. For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 5000USD * 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.

In [ ]:
# Create a new column about the year of planned expiration time
loans['planned_expiration_time_year']=pd.to_datetime(loans['planned_expiration_time']).dt.year
# Drop rows with missing values in 'planned_expiration_time_year' column and 'disburse_time_year' column
loans.dropna(subset=['planned_expiration_time_year'],inplace=True)
loans.dropna(subset=['disburse_time_year'],inplace=True)
# Remove rows where the year of disburse time is > than the year of planned expiration time
loans=loans[loans['disburse_time_year']<=loans['planned_expiration_time_year']]

time: 2.04 s


In [ ]:
loans['days_in_disburse_time_year']=365
loans['days_in_planned_expiration_time_year']=365
loans.loc[loans['disburse_time_year'] == 2012, 'days_in_disburse_time_year'] = 366
loans.loc[loans['disburse_time_year'] == 2016, 'days_in_disburse_time_year'] = 366
loans.loc[loans['planned_expiration_time'] == 2012, 'days_in_planned_expiration_time_year'] = 366
loans.loc[loans['planned_expiration_time'] == 2016, 'days_in_planned_expiration_time_year'] = 366

time: 171 ms


In [ ]:
# Created a new column with the number of days to be attributed to the first year
loans['days_disburse_time_year']= loans['days_in_disburse_time_year']- pd.to_datetime(loans['disburse_time']).dt.dayofyear
# Created a new column about the number of days of the relative year of planned expiration time
loans['days_planned_expiration_time_year']=pd.to_datetime(loans['planned_expiration_time']).dt.dayofyear
# Created a new column computing the difference in year between planned expiration time and disburse time
loans['difference_years']=loans['planned_expiration_time_year']-loans['disburse_time_year']

time: 416 ms


In [ ]:
# Created new columns for each year of our interest
bound = range(int(loans['disburse_time_year'].min()),int(loans['planned_expiration_time_year'].max())+1)
for i in bound:
    loans[str(i)] = 0

time: 19.9 ms


In [ ]:
# Selected only columns we need
atribute = ['loan_amount', '2011', '2012', '2013', '2014','2015', '2016', 
            '2017', '2018', 'difference_years', 'days_disburse_time_year',
            'days_planned_expiration_time_year', 'disburse_time_year', 'planned_expiration_time_year', 'days_in_disburse_time_year','days_in_planned_expiration_time_year']
loans = loans[atribute]
loans.head()

,loan_amount,2011,2012,2013,2014,2015,2016,2017,2018,difference_years,days_disburse_time_year,days_planned_expiration_time_year,disburse_time_year,planned_expiration_time_year,days_in_disburse_time_year,days_in_planned_expiration_time_year
0,125.0,0,0,0,0,0,0,0,0,1.0,9,45,2013.0,2014.0,365,365
1,400.0,0,0,0,0,0,0,0,0,1.0,11,85,2013.0,2014.0,365,365
2,400.0,0,0,0,0,0,0,0,0,0.0,356,46,2014.0,2014.0,365,365
3,625.0,0,0,0,0,0,0,0,0,0.0,348,52,2014.0,2014.0,365,365
4,425.0,0,0,0,0,0,0,0,0,1.0,14,44,2013.0,2014.0,365,365


time: 954 ms


In [ ]:
 loans['difference_years'].unique()

array([1., 0., 5., 4.])

time: 10.3 ms


In [ ]:
 # Create function 
def loan(x):
    if x['difference_years'] == 0: # se disburse e planned hanno stipulazione e scadenza nello stesso anno.
        x[str(int(x['disburse_time_year']))]= x['loan_amount'] # l'intero ammontare di loan_amount viene inserito 
                                                       # nella colonna in corrispondenza del medesimo anno.
    elif x['difference_years']== 1:
        x[str(int(x['disburse_time_year']))]= (x['days_disburse_time_year']*x['loan_amount'])/((x['days_disburse_time_year'])+ x['days_planned_expiration_time_year']+ ((x['difference_years']-1)*int(x['days_in_disburse_time_year'])))
        x[str(int(x['planned_expiration_time_year']))]= x['loan_amount'] - x[str(int(x['disburse_time_year']))]
        
    elif x['difference_years'] > 1:
        x[str(int(x['disburse_time_year']))]= x['days_disburse_time_year']*x['loan_amount'] / (x['days_disburse_time_year'] + x['days_planned_expiration_time_year'] + ((x['difference_years']-1)*int(x['days_in_disburse_time_year'])))
        x[str(int(x['planned_expiration_time_year']))]=(x['days_planned_expiration_time_year']*x['loan_amount'])/((x['days_disburse_time_year'])+ x['days_planned_expiration_time_year']+ ((x['difference_years']-1)* int(x['days_in_planned_expiration_time_year'])))
        
        for y in range(1,int(x['difference_years'])):
          if (int(x['disburse_time_year']+y) == 2012 or int(x['disburse_time_year']+y) == 2016) :
            x[str(int(x['disburse_time_year']+y))] = (366*x['loan_amount'])/((x['days_disburse_time_year'])+ x['days_planned_expiration_time_year']+ ((x['difference_years']-1)*366))
          else :
            x[str(int(x['disburse_time_year']+y))] = (365*x['loan_amount'])/((x['days_disburse_time_year'])+ x['days_planned_expiration_time_year']+ ((x['difference_years']-1)*365))

    return x

time: 10.4 ms


In [ ]:
# Apply function
df1 = loans.apply(loan,axis=1)
df1.head()

,loan_amount,2011,2012,2013,2014,2015,2016,2017,2018,difference_years,days_disburse_time_year,days_planned_expiration_time_year,disburse_time_year,planned_expiration_time_year,days_in_disburse_time_year,days_in_planned_expiration_time_year
0,125.0,0.0,0.0,20.833333,104.166667,0.0,0.0,0.0,0.0,1.0,9.0,45.0,2013.0,2014.0,365.0,365.0
1,400.0,0.0,0.0,45.833333,354.166667,0.0,0.0,0.0,0.0,1.0,11.0,85.0,2013.0,2014.0,365.0,365.0
2,400.0,0.0,0.0,0.000000,400.000000,0.0,0.0,0.0,0.0,0.0,356.0,46.0,2014.0,2014.0,365.0,365.0
3,625.0,0.0,0.0,0.000000,625.000000,0.0,0.0,0.0,0.0,0.0,348.0,52.0,2014.0,2014.0,365.0,365.0
4,425.0,0.0,0.0,102.586207,322.413793,0.0,0.0,0.0,0.0,1.0,14.0,44.0,2013.0,2014.0,365.0,365.0


time: 2min 31s


In [ ]:
# Show result
loan_amount_by_years = [(lambda x: str(x))(x) for x in list(bound)] 
df1[loan_amount_by_years].apply(lambda x: int(x.sum())).reset_index().rename(columns={'index':'year',0:'total'})

,year,total
0,2011,629911
1,2012,113274906
2,2013,124937120
3,2014,153151073
4,2015,159759915
5,2016,158987624
6,2017,171861891
7,2018,5705070


time: 70.6 ms
